<a href="https://colab.research.google.com/github/Hongyongmin/Edwith-Pytorch/blob/main/6_Softmax_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
torch.manual_seed(1)

In [ ]:
z = torch.FloatTensor([1,2,3])

In [ ]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)                  # 각각 1,2,3,일 확률

tensor([0.0900, 0.2447, 0.6652])


In [ ]:
hypothesis.sum()  # 확률의 합은 1

tensor(1.)

In [ ]:
z = torch.rand(3, 5, requires_grad=True)    # 랜덤하게 3x5 토치 생성
hypothesis = F.softmax(z, dim=1)             # softmax를 수행(2행에 대한 것)
print(hypothesis)      #예측값임

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [ ]:
y = torch.randint(5, (3,)).long()          # 랜덤하게 생성한 정답
print(y)              # y 가 0,2,1 이므로, 1행에서는 1번째 값이, 2행에서는 3번째 값이 정답임

tensor([0, 2, 1])


In [ ]:
y_one_hot = torch.zeros_like(hypothesis)
print(y_one_hot)
y_one_hot.scatter_(1, y.unsqueeze(1),1)  #언더바는 inplace를 의미

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [ ]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


In [ ]:
torch.log(F.softmax(z, dim=1))        ##소프트맥스 함수를 제공하기도 하고

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [ ]:
F.log_softmax(z, dim=1)       ##로그소프트맥스도 제공함

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

In [ ]:
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()   #소프트맥스로 써도 되고

tensor(1.4689, grad_fn=<MeanBackward0>)

In [ ]:
F.nll_loss(F.log_softmax(z, dim=1),y)  #로그소프트맥스도 된다. 
#nll은 Negative Log Likelihood의 약자로 한층 더 간단해진 형태

tensor(1.4689, grad_fn=<NllLossBackward>)

In [ ]:
F.cross_entropy(z,y)  #한 층 더 단순화한 형태

tensor(1.4689, grad_fn=<NllLossBackward>)

In [ ]:
x_train = [[1,2,1,1],
          [2,1,3,2],
          [3,1,3,4],
           [4,1,5,5],
           [1,7,5,5],
          [1,2,5,6],
           [1,6,6,6],
           [1,7,7,7]]
y_train = [2,2,2,1,1,1,0,0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [ ]:
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1),1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch %100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


In [ ]:
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W,b], lr= 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train) ##cost를 cross-entropy로 간단히 표현한것
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 ==0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [ ]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3)
        
    def forward(self,x):
        return self.linear(x)

In [ ]:
model = SoftmaxClassifierModel()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    prediction = model(x_train)  #train하는 과정도 model로 간단히 한것
    
    cost = F.cross_entropy(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609258
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269521
Epoch 1000/1000 Cost: 0.241922
